## 3.3. Generación de indicadores técnicos

A partir de los precios normalizados y del volumen de transacciones, se generaron una serie de **indicadores técnicos** destinados a enriquecer el conjunto de variables predictivas.  
Estos indicadores fueron seleccionados por su relevancia en el análisis financiero y su capacidad para reflejar tendencias, volatilidad y señales de cambio en el comportamiento del mercado.  

**• Media Móvil Simple (SMA)**

La **SMA (Simple Moving Average)** calcula el promedio de los precios de cierre durante una ventana temporal específica, suavizando las fluctuaciones a corto plazo y revelando tendencias generales del mercado.  
Su inclusión permite identificar con mayor claridad periodos de crecimiento sostenido o corrección.  
En este estudio se implementó la SMA para ventanas de **5, 10 y 20 días**, aplicadas a los precios de cierre normalizados de BBVA y Banco Santander.

**Fórmula:**

$
SMA_t = \frac{1}{n} \sum_{i=0}^{n-1} P_{t-i}
$

**Donde:**
- \(SMA_t\): Media móvil simple en el tiempo \(t\)  
- \(n\): Tamaño de la ventana temporal  
- \(P_{t-i}\): Precio de cierre en el día \(t - i\)  

Este indicador ofrece una visión depurada de la evolución del precio, eliminando el ruido diario y facilitando la identificación de la tendencia predominante en el periodo analizado (hasta el 5 de noviembre de 2025).


In [1]:
# =========================
# 0) Imports y rutas
# =========================
import pandas as pd
from pathlib import Path

IN_DIR  = Path("../data/processed")   # donde están los *_core_scaled.csv
OUT_DIR = Path("../data/processed")   # guardaremos aquí con SMA
OUT_DIR.mkdir(parents=True, exist_ok=True)

FILES = {
    "BBVA": IN_DIR / "BBVA_core_scaled.csv",
    "SAN" : IN_DIR / "SAN_core_scaled.csv",
}

# =========================
# 1) Utilidades
# =========================
def load_scaled(path: Path) -> pd.DataFrame:
    """Carga CSV, asegura Date como índice datetime ordenado."""
    df = pd.read_csv(path)
    if "Date" not in df.columns:
        raise ValueError(f"{path.name}: no se encontró columna 'Date'.")
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"]).sort_values("Date").set_index("Date")
    return df

def add_sma(df: pd.DataFrame, col: str = "Close", windows=(5, 10, 20)) -> pd.DataFrame:
    """Añade SMA_n sobre la columna col. Deja NaN en las primeras n-1 filas."""
    if col not in df.columns:
        raise ValueError(f"No existe la columna '{col}' en el DataFrame.")
    out = df.copy()
    for w in windows:
        out[f"SMA_{w}"] = out[col].rolling(window=w, min_periods=w).mean()
    return out

# =========================
# 2) Proceso por ticker
# =========================
results = {}
for name, path in FILES.items():
    df = load_scaled(path)
    df_sma = add_sma(df, col="Close", windows=(5, 10, 20))

    out_path = OUT_DIR / f"{name}_with_SMA.csv"
    df_sma.to_csv(out_path, index=True)

    results[name] = df_sma
    print(f"{name}: guardado -> {out_path}")
    # vista rápida de las últimas 3 filas
    print(df_sma[["Close", "SMA_5", "SMA_10", "SMA_20"]].tail(3), "\n")

# =========================
# 3) (Opcional) Comprobación de NaNs iniciales
# =========================
for name, df in results.items():
    na_counts = df[["SMA_5", "SMA_10", "SMA_20"]].isna().sum()
    print(f"{name}: NaNs iniciales por ventana (esperado = w-1 si empieza al principio):")
    print(na_counts, "\n")


BBVA: guardado -> ..\data\processed\BBVA_with_SMA.csv
               Close     SMA_5    SMA_10    SMA_20
Date                                              
2025-10-28  0.877319  0.869147  0.853183  0.834679
2025-10-29  0.900084  0.875918  0.863660  0.838196
2025-10-30  0.882572  0.879654  0.872795  0.840968 

SAN: guardado -> ..\data\processed\SAN_with_SMA.csv
               Close     SMA_5    SMA_10    SMA_20
Date                                              
2025-10-28  0.619817  0.606558  0.605568  0.614960
2025-10-29  0.652227  0.616828  0.609886  0.615651
2025-10-30  0.632242  0.623015  0.611372  0.616033 

BBVA: NaNs iniciales por ventana (esperado = w-1 si empieza al principio):
SMA_5      4
SMA_10     9
SMA_20    19
dtype: int64 

SAN: NaNs iniciales por ventana (esperado = w-1 si empieza al principio):
SMA_5      4
SMA_10     9
SMA_20    19
dtype: int64 



**• Media Móvil Exponencial (EMA)**

La **EMA (Exponential Moving Average)**, al igual que la media móvil simple, permite identificar las tendencias generales del mercado.  
Sin embargo, a diferencia de la SMA, la EMA asigna un **mayor peso a los valores más recientes**, lo que la hace más sensible a los cambios rápidos y adecuada para detectar giros de tendencia con menor retraso temporal.

En este estudio, la EMA se calculó para una **ventana de 10 días**, aplicada sobre los precios de cierre normalizados de BBVA y Banco Santander.

**Fórmula:**

$
EMA_t = \alpha \cdot P_t + (1 - \alpha) \cdot EMA_{t-1}
$

**Donde:**
- $\alpha = \dfrac{2}{n+1}$: Factor de suavizado  
- $EMA_t$: Media móvil exponencial en el tiempo \(t\)  
- $P_t$: Precio de cierre en el instante \(t\)  
- $n$: Tamaño de la ventana temporal (en este caso, 10 días)  

La EMA proporciona una visión más dinámica de la evolución del precio, otorgando prioridad a la información más reciente y mejorando la capacidad del modelo para adaptarse a movimientos bruscos o tendencias emergentes.


In [4]:
# =========================
# 0) Imports y rutas
# =========================
import pandas as pd
from pathlib import Path

IN_DIR  = Path("../data/processed/SMA")
OUT_DIR = Path("../data/processed/SMA")
OUT_DIR.mkdir(parents=True, exist_ok=True)

FILES = {
    "BBVA": IN_DIR / "BBVA_with_SMA.csv",
    "SAN" : IN_DIR / "SAN_with_SMA.csv",
}

# =========================
# 1) Función EMA
# =========================
def add_ema(df: pd.DataFrame, col: str = "Close", span: int = 10) -> pd.DataFrame:
    """
    Añade la Media Móvil Exponencial (EMA) a un DataFrame.
    Parámetros:
      - col: columna sobre la que se aplica la EMA
      - span: número de días de la ventana
    """
    out = df.copy()
    out[f"EMA_{span}"] = out[col].ewm(span=span, adjust=False).mean()
    return out

# =========================
# 2) Generar y guardar
# =========================
results = {}
for name, path in FILES.items():
    # Leer CSV con índice de fechas
    df = pd.read_csv(path, parse_dates=["Date"], index_col="Date")

    # Calcular EMA con ventana de 10 días
    df_ema = add_ema(df, col="Close", span=10)

    # Guardar resultado
    out_path = OUT_DIR / f"{name}_with_EMA.csv"
    df_ema.to_csv(out_path, index=True)

    # Almacenar en diccionario y mostrar resumen
    results[name] = df_ema
    print(f"{name}: guardado -> {out_path}")
    print(df_ema[["Close", "EMA_10"]].tail(3), "\n")

# =========================
# 3) (Opcional) Validar
# =========================
for name, df in results.items():
    print(f"{name} - primeras observaciones con EMA_10 calculada:")
    print(df[["Close", "EMA_10"]].head(12), "\n")


BBVA: guardado -> ..\data\processed\SMA\BBVA_with_EMA.csv
               Close    EMA_10
Date                          
2025-10-28  0.877319  0.857223
2025-10-29  0.900084  0.865016
2025-10-30  0.882572  0.868208 

SAN: guardado -> ..\data\processed\SMA\SAN_with_EMA.csv
               Close    EMA_10
Date                          
2025-10-28  0.619817  0.609131
2025-10-29  0.652227  0.616967
2025-10-30  0.632242  0.619744 

BBVA - primeras observaciones con EMA_10 calculada:
               Close    EMA_10
Date                          
2000-01-03  0.669129  0.669129
2000-01-04  0.648380  0.665357
2000-01-05  0.630995  0.659109
2000-01-06  0.630995  0.653998
2000-01-07  0.640529  0.651549
2000-01-10  0.628191  0.647302
2000-01-11  0.613050  0.641074
2000-01-12  0.604638  0.634449
2000-01-13  0.586692  0.625766
2000-01-14  0.600712  0.621211
2000-01-17  0.593982  0.616260
2000-01-18  0.578841  0.609457 

SAN - primeras observaciones con EMA_10 calculada:
               Close    EMA_10
Da

**• Índice de Fuerza Relativa (RSI)**

El **RSI (Relative Strength Index)** mide la velocidad y magnitud de los cambios de precio para identificar zonas de **sobrecompra** (RSI > 70) y **sobreventa** (RSI < 30).  
Introducido por J. Welles Wilder en *New Concepts in Technical Trading Systems*, el periodo estándar de cálculo es de **14 días**.

**Fórmula:**


$RSI = 100 - \frac{100}{1 + RS}$


**Donde:**
-  $RS = \frac{\text{Ganancia Promedio}}{\text{Pérdida Promedio}} $, calculadas como medias móviles suavizadas (método de Wilder) de los últimos \(n\) días.
-  $n = 14$  días en este estudio.

El RSI proporciona una señal compacta del **ímpetu** del precio: valores altos indican presión compradora sostenida, mientras que valores bajos reflejan predominio de presiones vendedoras. Su naturaleza acotada entre 0 y 100 facilita la interpretación y la combinación con otros indicadores técnicos (SMA, EMA).


In [6]:
# =========================
# 0) Imports y rutas
# =========================
import pandas as pd
from pathlib import Path

IN_DIR  = Path("../data/processed/EMA")     # donde guardaste *_with_EMA.csv
OUT_DIR = Path("../data/processed/INDICATORS")
OUT_DIR.mkdir(parents=True, exist_ok=True)

FILES = {
    "BBVA": IN_DIR / "BBVA_with_EMA.csv",
    "SAN" : IN_DIR / "SAN_with_EMA.csv",
}

# =========================
# 1) Función RSI (Wilder)
# =========================
def add_rsi(df: pd.DataFrame, col: str = "Close", window: int = 14) -> pd.DataFrame:
    """
    RSI de J. W. Wilder (periodo estándar 14).
    Usa medias móviles exponenciales con alpha = 1/window (RMA/Wilder EMA).
    """
    out = df.copy()
    delta = out[col].diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    # Promedios suavizados de ganancias y pérdidas (Wilder)
    avg_gain = gain.ewm(alpha=1/window, adjust=False, min_periods=window).mean()
    avg_loss = loss.ewm(alpha=1/window, adjust=False, min_periods=window).mean()

    rs = avg_gain / avg_loss
    out[f"RSI_{window}"] = 100 - (100 / (1 + rs))
    return out

# =========================
# 2) Generar y guardar
# =========================
results = {}
for name, path in FILES.items():
    df = pd.read_csv(path, parse_dates=["Date"], index_col="Date")

    df_rsi = add_rsi(df, col="Close", window=14)

    out_path = OUT_DIR / f"{name}_with_RSI.csv"
    df_rsi.to_csv(out_path, index=True)
    results[name] = df_rsi

    print(f"{name}: guardado -> {out_path}")
    print(df_rsi[["Close", "EMA_10", "RSI_14"]].tail(3), "\n")

# =========================
# 3) (Opcional) Chequeos rápidos
# =========================
for name, df in results.items():
    # Al principio habrá NaNs (13 filas) por la ventana de 14
    print(f"{name} - NaNs RSI_14:", df["RSI_14"].isna().sum())
    # Rango esperado 0..100
    print(f"{name} - rango RSI_14:", float(df["RSI_14"].min()), "→", float(df["RSI_14"].max()), "\n")


BBVA: guardado -> ..\data\processed\INDICATORS\BBVA_with_RSI.csv
               Close    EMA_10     RSI_14
Date                                     
2025-10-28  0.877319  0.857223  63.275547
2025-10-29  0.900084  0.865016  67.904567
2025-10-30  0.882572  0.868208  61.484491 

SAN: guardado -> ..\data\processed\INDICATORS\SAN_with_RSI.csv
               Close    EMA_10     RSI_14
Date                                     
2025-10-28  0.619817  0.609131  54.342585
2025-10-29  0.652227  0.616967  64.795094
2025-10-30  0.632242  0.619744  56.244555 

BBVA - NaNs RSI_14: 14
BBVA - rango RSI_14: 7.214239103717858 → 86.56173324601636 

SAN - NaNs RSI_14: 14
SAN - rango RSI_14: 12.58951071783666 → 89.61683966053022 



**• Volatilidad Histórica**

La **volatilidad histórica** refleja la variabilidad del precio de una acción, estimando el riesgo asociado a su comportamiento pasado.  
Se calcula como la desviación estándar de los precios de cierre en una ventana móvil, lo que permite cuantificar la intensidad de las fluctuaciones del mercado.

En este análisis se consideró una **ventana de 10 días**, aplicada sobre los precios de cierre normalizados de BBVA y Banco Santander.

**Fórmula:**


$\sigma_t = \sqrt{\frac{\sum_{i=0}^{n-1}(P_{t-i} - \mu)^2}{n}}$


**Donde:**
- $\sigma_t$: Volatilidad histórica en el tiempo \(t\)  
- $\mu$: Media de los precios en la ventana  
- $n = 10$: Tamaño de la ventana temporal  

Este indicador se utiliza habitualmente para medir el **riesgo histórico** de un activo financiero:  
valores altos de $\sigma_t$ implican mayor inestabilidad en el precio, mientras que valores bajos reflejan un comportamiento más estable.


In [7]:
# =========================
# 0) Imports y rutas
# =========================
import pandas as pd
from pathlib import Path

IN_DIR  = Path("../data/processed/RSI")   # donde guardaste los *_with_RSI.csv
OUT_DIR = Path("../data/processed/INDICATORS")
OUT_DIR.mkdir(parents=True, exist_ok=True)

FILES = {
    "BBVA": IN_DIR / "BBVA_with_RSI.csv",
    "SAN" : IN_DIR / "SAN_with_RSI.csv",
}

# =========================
# 1) Función de Volatilidad Histórica
# =========================
def add_volatility(df: pd.DataFrame, col: str = "Close", window: int = 10) -> pd.DataFrame:
    """
    Calcula la volatilidad histórica (desviación estándar) del precio de cierre
    en una ventana móvil de tamaño 'window' (por defecto, 10 días).
    """
    out = df.copy()
    out[f"Volatility_{window}"] = out[col].rolling(window=window, min_periods=window).std()
    return out

# =========================
# 2) Generar y guardar
# =========================
results = {}
for name, path in FILES.items():
    df = pd.read_csv(path, parse_dates=["Date"], index_col="Date")

    # Calcular volatilidad histórica
    df_vol = add_volatility(df, col="Close", window=10)

    out_path = OUT_DIR / f"{name}_with_VOL.csv"
    df_vol.to_csv(out_path, index=True)
    results[name] = df_vol

    print(f"{name}: guardado -> {out_path}")
    print(df_vol[["Close", "Volatility_10"]].tail(3), "\n")

# =========================
# 3) (Opcional) Verificación rápida
# =========================
for name, df in results.items():
    print(f"{name} - NaNs iniciales Volatility_10:", df["Volatility_10"].isna().sum())
    print(f"{name} - rango de volatilidad:", float(df["Volatility_10"].min()), "→", float(df["Volatility_10"].max()), "\n")


BBVA: guardado -> ..\data\processed\INDICATORS\BBVA_with_VOL.csv
               Close  Volatility_10
Date                               
2025-10-28  0.877319       0.033282
2025-10-29  0.900084       0.029291
2025-10-30  0.882572       0.014897 

SAN: guardado -> ..\data\processed\INDICATORS\SAN_with_VOL.csv
               Close  Volatility_10
Date                               
2025-10-28  0.619817       0.008752
2025-10-29  0.652227       0.017217
2025-10-30  0.632242       0.018527 

BBVA - NaNs iniciales Volatility_10: 9
BBVA - rango de volatilidad: 0.0015416025559739997 → 0.06414201001457895 

SAN - NaNs iniciales Volatility_10: 9
SAN - rango de volatilidad: 0.0013371941843138891 → 0.08646386409111721 



### • Rango Verdadero Promedio (ATR)

El **ATR (Average True Range)** mide la volatilidad diaria del precio teniendo en cuenta los máximos, mínimos y cierres. Es especialmente útil en entornos de alta fluctuación para evaluar el riesgo y definir niveles de entrada/salida en estrategias de trading.  
En este trabajo se calculó con una **ventana de 14 días**.

**Cálculo del Rango Verdadero (TR):**
$
TR_t = \max\left(
\text{High}_t - \text{Low}_t,\;
|\text{High}_t - \text{Close}_{t-1}|,\;
|\text{Low}_t - \text{Close}_{t-1}|
\right)
$

**Fórmula del ATR:**
$
ATR_t = \frac{1}{n}\sum_{i=0}^{n-1} TR_{t-i}
$
con $n = 14$.

El ATR ofrece una medida robusta de la **amplitud típica de movimiento** del precio: valores elevados indican mayor volatilidad reciente y, por tanto, mayor riesgo operativo; valores bajos reflejan un entorno de oscilaciones más contenidas.


In [8]:
# =========================
# 0) Imports y rutas
# =========================
import pandas as pd
from pathlib import Path

DIR_IND  = Path("../data/processed/INDICATORS")
DIR_SMA  = Path("../data/processed/EMA")
DIR_BASE = Path("../data/processed")
OUT_DIR  = DIR_IND
OUT_DIR.mkdir(parents=True, exist_ok=True)

TICKERS = ["BBVA", "SAN"]
WINDOW = 14

# =========================
# 1) Carga robusta (elige el archivo más reciente que tenga High/Low/Close)
# =========================
def load_best(ticker: str) -> pd.DataFrame:
    candidates = [
        DIR_IND / f"{ticker}_with_VOL.csv",
        DIR_IND / f"{ticker}_with_RSI.csv",
        DIR_SMA / f"{ticker}_with_EMA.csv",
        DIR_SMA / f"{ticker}_with_SMA.csv",
        DIR_BASE / f"{ticker}_core_scaled.csv",
    ]
    for p in candidates:
        if p.exists():
            df = pd.read_csv(p, parse_dates=["Date"], index_col="Date").sort_index()
            if all(c in df.columns for c in ["High", "Low", "Close"]):
                return df
    raise FileNotFoundError(f"No se encontró un CSV válido con High/Low/Close para {ticker}.")

# =========================
# 2) ATR (Wilder): TR_t = max( High-Low, |High-Close_{t-1}|, |Low-Close_{t-1}| )
# =========================
def add_atr(df: pd.DataFrame, window: int = 14) -> pd.DataFrame:
    out = df.copy()
    prev_close = out["Close"].shift(1)
    tr = pd.concat([
        (out["High"] - out["Low"]).abs(),
        (out["High"] - prev_close).abs(),
        (out["Low"]  - prev_close).abs()
    ], axis=1).max(axis=1)

    out[f"ATR_{window}"] = tr.rolling(window=window, min_periods=window).mean()
    return out

# =========================
# 3) Ejecutar por ticker y guardar
# =========================
results = {}
for t in TICKERS:
    df = load_best(t)
    df_atr = add_atr(df, window=WINDOW)
    out_path = OUT_DIR / f"{t}_with_ATR.csv"
    df_atr.to_csv(out_path, index=True)
    results[t] = df_atr
    print(f"{t}: guardado -> {out_path}")
    print(df_atr[["High","Low","Close", f"ATR_{WINDOW}"]].tail(3), "\n")

# =========================
# 4) Chequeos rápidos (opcionales)
# =========================
for t, df in results.items():
    print(f"{t} - NaNs iniciales ATR_{WINDOW}:", df[f"ATR_{WINDOW}"].isna().sum())
    mn, mx = float(df[f"ATR_{WINDOW}"].min(skipna=True)), float(df[f"ATR_{WINDOW}"].max(skipna=True))
    print(f"{t} - rango ATR_{WINDOW}: {mn} → {mx}\n")


BBVA: guardado -> ..\data\processed\INDICATORS\BBVA_with_ATR.csv
               High       Low     Close    ATR_14
Date                                             
2025-10-28  0.86853  0.875177  0.877319  0.015290
2025-10-29  0.88997  0.876057  0.900084  0.015336
2025-10-30  0.87983  0.875764  0.882572  0.016224 

SAN: guardado -> ..\data\processed\INDICATORS\SAN_with_ATR.csv
                High       Low     Close    ATR_14
Date                                              
2025-10-28  0.611393  0.613532  0.619817  0.009506
2025-10-29  0.640933  0.619895  0.652227  0.010080
2025-10-30  0.638364  0.624514  0.632242  0.011618 

BBVA - NaNs iniciales ATR_14: 13
BBVA - rango ATR_14: 0.002573426305815443 → 0.03999269718244106

SAN - NaNs iniciales ATR_14: 13
SAN - rango ATR_14: 0.0023475405060940572 → 0.05631763711519681



**• Normalización y limpieza final de los indicadores técnicos**

Tras el preprocesamiento de los datos originales y la inclusión de los indicadores técnicos, se generaron valores nulos en las columnas de dichos indicadores debido a los cálculos basados en ventanas.  
Estas filas iniciales, sin datos completos para realizar los cálculos, fueron eliminadas para mantener la consistencia del conjunto de datos.

Una vez depurados los valores nulos, todos los indicadores técnicos fueron **normalizados mediante el método de escalado Min-Max**, garantizando que los valores quedaran dentro del rango [0, 1].  
Este procedimiento asegura que los indicadores (**SMA₅**, **EMA₁₀**, **RSI₁₄**, **Volatilidad₁₀**, **ATR₁₄**, etc.) sean directamente comparables entre sí y facilita su uso en modelos predictivos.  

Además, la normalización previene que indicadores con rangos más amplios —como la volatilidad o el ATR— tengan una influencia desproporcionada sobre aquellos con variaciones más pequeñas, asegurando un equilibrio adecuado en la modelización.


In [11]:
# ============================================
# 0) Imports y rutas
# ============================================
import json
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

IN_DIR  = Path("../data/processed/Technical_indicators")     # donde están *_with_ATR.csv
OUT_DIR = Path("../data/processed/ready_for_modeling")  # salida final
OUT_DIR.mkdir(parents=True, exist_ok=True)

TICKERS = ["BBVA", "SAN"]

# Columnas que mantendremos (ajústalo si necesitas menos/más)
KEEP_COLS = [
    "Adj Close","Close","Dividends","High","Low","Open","Stock Splits","Volume","Dividends_bin",
    "SMA_5","SMA_10","SMA_20","EMA_10","RSI_14","Volatility_10","ATR_14"
]

# Columnas a escalar Min-Max (incluye RSI_14 y el resto de indicadores/base numéricos)
TO_SCALE = [
    "Adj Close","Close","High","Low","Open","Volume",
    "SMA_5","SMA_10","SMA_20","EMA_10","RSI_14","Volatility_10","ATR_14"
]
# Nota: dejamos sin escalar "Dividends", "Stock Splits" y "Dividends_bin" (eventos / binaria)


# ============================================
# 1) Función de limpieza + escalado
# ============================================
def clean_and_minmax(df: pd.DataFrame, to_scale: list[str]) -> tuple[pd.DataFrame, dict]:
    df = df.copy()
    # 1) eliminar NaN por ventanas (SMA/EMA/RSI/Vol/ATR)
    df = df.dropna()

    # 2) asegurar que existen todas las columnas a escalar
    missing = [c for c in to_scale if c not in df.columns]
    if missing:
        raise ValueError(f"Faltan columnas para escalar: {missing}")

    # 3) Min-Max sobre las columnas indicadas (incluye RSI_14)
    scaler = MinMaxScaler()
    df[to_scale] = scaler.fit_transform(df[to_scale])

    # Guardamos min/max para reproducibilidad (solo de las columnas escaladas)
    params = {col: {"min": float(scaler.data_min_[i]), "max": float(scaler.data_max_[i])}
              for i, col in enumerate(to_scale)}
    return df, params


# ============================================
# 2) Proceso por ticker
# ============================================
all_params = {}
for t in TICKERS:
    # Cargar el dataset más completo (con ATR)
    path = IN_DIR / f"{t}_with_ATR.csv"
    df = pd.read_csv(path, parse_dates=["Date"], index_col="Date").sort_index()

    # Filtrar/ordenar columnas si procede
    cols_presentes = [c for c in KEEP_COLS if c in df.columns]
    df = df[cols_presentes]

    # Limpiar y escalar (RSI_14 incluido)
    df_scaled, params = clean_and_minmax(df, TO_SCALE)

    # Guardar CSV final
    out_csv = OUT_DIR / f"{t}_final_ready.csv"
    df_scaled.to_csv(out_csv, index=True)
    print(f"✅ {t}: guardado -> {out_csv}")

    # Verificación: rangos REALES tras el escalado (0..1)
    min_after = df_scaled[TO_SCALE].min().rename("min_after")
    max_after = df_scaled[TO_SCALE].max().rename("max_after")
    check = pd.concat([min_after, max_after], axis=1)
    print(f"\n{t} - Rangos tras Min-Max (esperado 0..1) solo en columnas escaladas:\n{check}\n")

    all_params[t] = params

# ============================================
# 3) Guardar parámetros de escalado (min/max por columna)
# ============================================
params_path = OUT_DIR / "minmax_params.json"
with open(params_path, "w", encoding="utf-8") as f:
    json.dump(all_params, f, indent=2, ensure_ascii=False)
print("Parámetros guardados en:", params_path)


✅ BBVA: guardado -> ..\data\processed\ready_for_modeling\BBVA_final_ready.csv

BBVA - Rangos tras Min-Max (esperado 0..1) solo en columnas escaladas:
               min_after  max_after
Adj Close            0.0        1.0
Close                0.0        1.0
High                 0.0        1.0
Low                  0.0        1.0
Open                 0.0        1.0
Volume               0.0        1.0
SMA_5                0.0        1.0
SMA_10               0.0        1.0
SMA_20               0.0        1.0
EMA_10               0.0        1.0
RSI_14               0.0        1.0
Volatility_10        0.0        1.0
ATR_14               0.0        1.0

✅ SAN: guardado -> ..\data\processed\ready_for_modeling\SAN_final_ready.csv

SAN - Rangos tras Min-Max (esperado 0..1) solo en columnas escaladas:
               min_after  max_after
Adj Close            0.0        1.0
Close                0.0        1.0
High                 0.0        1.0
Low                  0.0        1.0
Open              